In [117]:
# run mean OPA 
import intake 
import os
# add directory where the utils.py file is located
import xarray as xr 
import importlib
import dask as dk
import numpy as np 

# for levante
os.chdir('/home/b/b382291/git/one_pass/src')
imported_module = importlib.import_module("opa")

os.chdir('/home/b/b382291/git/one_pass/src')
importlib.reload(imported_module)
from opa import *
from opa import opa # from module (file.py) import class name 

os.chdir('/home/b/b382291/regridder/AQUA')
from aqua import Reader
from aqua.reader import catalogue


# initalising the statistic 
stat1 = opa(statistic = "min", statFreq = "daily", outputFreq = "weekly", save = True)

#What options can go in: 
#1. statistic: mean, var, std, min, max, threshExceed, hist, percentile
#2. statFreq: hourly, 3 hourly, 6 hourly, daily, monthly, anually,
#3. outputFreq: same as above. If this frequency is higher it will append to a dataSet a rolling time window of the requested statistic and output in memory
#4. save: True or False, do you want to save to netCDF or not 
#5. saveFreq: only required if save = True, how often do you want to save the data, if not set will equal outputFreq


# list of possible statistics: 

Running the algorithm

In [118]:

#for i in range(288, 288+24*28, 1): 
#for i in range(31,31 + 2*24*28, 1): 

for i in range(12*24*2, 12*24*2 + 7*24*2, 1): 

    # extract moving window 
    ds = tasr.isel(time=slice(i,i+1)) # slice(start, stop, step), extract 'moving window' which is hourly data
    #ds = ds.isel(time = slice(0,1),lat = slice(0,2), lon = slice(0,2))

    dm = stat1.compute(ds)



/home/b/b382291/.conda/envs/aqua/lib/python3.10/site-packages/xarray/conventions.py:224: SerializationWarning: variable timings has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(


ValueError: unable to infer dtype on variable 'timings'; xarray cannot serialize arbitrary Python objects

In [116]:
dm

<xarray.Dataset>
Dimensions:  (time: 7, lat: 900, lon: 1800)
Coordinates:
  * time     (time) datetime64[ns] 2020-02-01 2020-02-02 ... 2020-02-07
  * lon      (lon) float64 0.0 0.2 0.4 0.6 0.8 ... 359.0 359.2 359.4 359.6 359.8
  * lat      (lat) float64 -89.9 -89.7 -89.5 -89.3 -89.1 ... 89.3 89.5 89.7 89.9
Data variables:
    tas      (time, lat, lon) float64 dask.array<chunksize=(1, 900, 1800), meta=np.ndarray>
    timings  (time, lat, lon) object dask.array<chunksize=(1, 900, 1800), meta=np.ndarray>
Attributes:
    CDI_grid_type:                unstructured
    cell_methods:                 t: point
    height:                       2.0 m
    long_name:                    temperature in 2m
    number_of_grid_in_reference:  1
    param:                        0.0.0
    standard_name:                tas
    units:                        K
    OPA:                          daily min calculated using one-pass algorithm

In [2]:
#catalogue();
!pwd
# IFS uses GRIB 1 
# ICON is netCDF? 
#reader = Reader(model="IFS", exp="tco2559-ng5", source="ICMGG_atm2d")
#reader = Reader(model="IFS", exp="tco2559-ng5", source="ICMGG_atm2d", regrid="r020")
reader = Reader(model="ICON", exp="ngc2009", source="atm_2d_ml_R02B09", regrid="r020")

data = reader.retrieve(fix=False)
tasr = reader.regrid(data["tas"][:,:])

#tasr = reader.regrid(data["2t"][0:16,:])

/home/b/b382291/regridder/AQUA


In [180]:
# checking the output against np.mean 

ds2 = tasr.isel(time=slice(0,24)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
#npStat = np.mean(ds2, axis = axNum, dtype = np.float64)
#npStat = np.var(ds2, axis = axNum, dtype = np.float64, ddof = 1)
npStat = np.amax(ds2, axis = axNum)
npStat = np.squeeze(npStat)

In [182]:
a= np.squeeze(stat1.maxCum) - npStat
a.values
#np.squeeze(stat1.minCum.compute())

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
loadPath = "/home/b/b382291/git/data/"
files = os.listdir(loadPath)
files = sorted(files)
savedData = xr.open_dataset(loadPath + files[0])


ds2 = tasr.isel(time=slice(0,24)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
npMean = np.mean(ds2, axis = axNum, dtype = np.float64)
npMean = np.squeeze(npMean)

a = savedData.meanVar - npMean
a.values


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [17]:
ds2 = tasr.isel(time=slice(24,30)) # slice(start, stop, step), extract 'moving window' which is hourly data

axNum = ds2.get_axis_num('time')
npMean = np.mean(ds2, axis = axNum, dtype = np.float64)
npMean = np.squeeze(npMean)

a = meanStat.meanCum - npMean


NameError: name 'tasr' is not defined

In [53]:
#a = np.array([[[4, 5, 6, 7],[1,10,9,13]],[[0,0,4,0],[0,0,4,0]]])#
#b = np.array([[[8, 8, 8, 2],[1,7,10,13]],[[0,3,0,1],[0,3,0,1]]])
#a = da.from_array(a)
#b = da.from_array(b)

a = xr.DataArray(np.random.rand(1,900,1800))
b = np.random.rand(1,900,1800)
#a = da.from_array(a)
#b = da.from_array(b)

f = (da.asarray(ds))
e = (da.asarray(stat1.minCum))


def _complex_operation(my_arr1, my_arr2):
    fIndex = my_arr1 < my_arr2
    fIndex = fIndex.astype(np.int8)
    my_arr1[fIndex==0] = my_arr2[fIndex==0]
    return my_arr1 


tempOut = _complex_operation(a,b)

#delayed_result = dask.delayed(_complex_operation)(f,e)
# to create a dask array to use in the future
#my_new_arr = da.from_delayed(delayed_result, dtype=ds.dtype, shape=e.shape)
#cat = my_new_arr.compute()

IndexError: 3-dimensional boolean indexing is not supported. 

In [260]:

#minCum = da.zeros((1, np.size(ds.lat), np.size(ds.lon)))
#dsTime = pd.datetime(np.shape(ds))
dsTime = np.zeros(np.shape(ds))

dsTime = xr.zeros_like(minCum)

#dsTime([:,:,:]) = ds.time.values[0]

for i in range(0, 24, 1): 
    ds = tasr.isel(time=slice(i,i+1)) # slice(start, stop, step), extract 'moving window' which is hourly data
    ds = ds.isel(time = slice(0,1),lat = slice(0,1), lon = slice(0,1))

    a = pd.to_datetime(ds.time.values[0]) # this will be a individual value (one time point)

    if (i== 0):
        
        #dsTime = np.where(dsTime == 0, a, 0)
        dsTime = dsTime.where(dsTime != 0, a)

    elif (i > 0):
        minCum['time'] = ds.time
        dsTime['time'] = ds.time
        dsTime = dsTime.where(ds > minCum, a) # replace if false 
        ds = ds.where(ds < minCum, minCum) # replace if false 

    minCum = ds


In [252]:
dsTime.values

array([[[Timestamp('2020-01-20 01:00:00')]]], dtype=object)

In [251]:
ds.values

array([[[242.07190643]]])

In [286]:

dsDay = tasr.isel(time=slice(0,24)) # slice(start, stop, step), extract 'moving window' which is hourly data
dsDay = dsDay.isel(lat = slice(0,2), lon = slice(0,2))
ds = np.amin(dsDay, axis = 0, keepdims = True)
ds.values

array([[[242.07190643, 242.07190643],
        [242.09762343, 242.09762343]]])

In [116]:
dm1

<xarray.Dataset>
Dimensions:  (time: 1, lat: 900, lon: 1800)
Coordinates:
  * time     (time) object 2020-01-20
  * lon      (lon) float64 0.0 0.2 0.4 0.6 0.8 ... 359.0 359.2 359.4 359.6 359.8
  * lat      (lat) float64 -89.9 -89.7 -89.5 -89.3 -89.1 ... 89.3 89.5 89.7 89.9
Data variables:
    tas      (time, lat, lon) float64 dask.array<chunksize=(1, 900, 1800), meta=np.ndarray>
    timings  (time, lat, lon) object dask.array<chunksize=(1, 900, 1800), meta=np.ndarray>
Attributes:
    CDI_grid_type:                unstructured
    cell_methods:                 t: point
    height:                       2.0 m
    long_name:                    temperature in 2m
    number_of_grid_in_reference:  1
    param:                        0.0.0
    standard_name:                tas
    units:                        K
    OPA:                          daily max calculated using one-pass algorithm

In [51]:

if (hasattr(stat1, 'countAppend') == False or a == 2):
    b = 2
else:
    b = 3

In [48]:
TS.to_datetime64().astype('datetime64[D]')

numpy.datetime64('2020-01-26')